# Pandas Part 3

**Outline:**
- Combining and Merging Datasets
- GroupBy Operations

---

In [ ]:
# Import Packages
import pandas as pd
import numpy as np

## Combining Datasets

Pandas has a function, `pd.concat()`, which has a similar syntax to `np.concatenate`

In [ ]:
ser1 = pd.Series(['A', 'B', 'C'], index = [1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index = [4, 5, 6])

In [ ]:
ser1

In [ ]:
ser2

In [ ]:
pd.concat([ser1, ser2])

It also works to concatenate higher-dimensional objects, such as DataFrames:

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), columns = ['A', 'B'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), columns = ['A', 'B'])

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.concat([df1, df2])

One important difference between `np.concatenate` and `pd.concat` is that Pandas
concatenation *preserves indices*, even if the result will have duplicate indices!

**Ignoring the index**. Sometimes the index itself does not matter, and you would prefer it to simply be ignored. You can specify this option using the `ignore_index` flag. With this set to `True`, the concatenation will create a new integer index for the resulting Series:

In [ ]:
pd.concat([df1, df2], ignore_index = True)

By default, the concatenation takes place row-wise within the DataFrame (i.e.,
`axis = 0`).

Like `np.concatenate`, `pd.concat` allows specification of an axis along which concatenation will take place. Consider the following example:

In [ ]:
df3 = pd.DataFrame(np.arange(4).reshape(2, 2), columns = ['A', 'B'])
df4 = pd.DataFrame(3 + np.arange(4).reshape(2, 2), columns = ['C', 'D'])

In [ ]:
df3

In [ ]:
df4

In [ ]:
pd.concat([df3, df4], axis = 1)

## Merging Datasets

Consider the following two DataFrames, which contain information on several employees in a company:

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})

df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

In [ ]:
df1

In [ ]:
df2

To combine this information into a single DataFrame, we can use the `pd.merge()`
function:

In [ ]:
df3 = pd.merge(df1, df2)
df3

1.   `pd.merge()` function recognizes that each DataFrame has an "employee" column, and automatically joins using this column as a key.
2. Order of the "employee" column differs between `df1` and `df2`, and the `pd.merge()` function correctly accounts for this.



In [ ]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
df4

In [ ]:
df3

In [ ]:
df5 = pd.merge(df3, df4)
df5

The resulting DataFrame has an additional column with the “supervisor” information,
where the information is repeated in one or more locations as required by the inputs.

### Specification of the Merge Key
We’ve already seen the default behavior of `pd.merge()`: it looks for one or more matching column names between the two inputs, and uses this as the key.

However, often the column names will not match so nicely, and `pd.merge()` provides a variety of options for handling this.

**The on keyword**

You can explicitly specify the name of the key column using the on keyword,
which takes a column name or a list of column names:

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.merge(df1, df2, on = 'employee')

This option works only if both the left and right DataFrames have the specified column
name.

**The left_on and right_on keywords**

At times you may wish to merge two datasets with different column names; for example, we may have a dataset in which the employee name is labeled as “name” rather than “employee”.

In this case, we can use the left_on and right_on keywords to specify the two column names:

In [ ]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display(df1); display(df3)

In [ ]:
pd.merge(df1, df3, left_on = "employee", right_on = "name")

The result has a redundant column that we can drop if desired—for example, by
using the `drop()` method of DataFrames:

In [ ]:
pd.merge(df1, df3, left_on = "employee", right_on = "name").drop('name', axis = 1)

**The left_index and right_index keywords**

Sometimes, rather than merging on a column, you would instead like to merge on an index. For example, your data might look like this:

In [ ]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')

In [ ]:
df1a

In [ ]:
df2a

You can use the index as the key for merging by specifying the `left_index` and/or `right_index` flags in `pd.merge()`:

In [ ]:
pd.merge(df1a, df2a, left_index = True, right_index = True)

If you’d like to mix indices and columns, you can combine `left_index` with `right_on` or `left_on` with `right_index` to get the desired behavior:

In [ ]:
df1a

In [ ]:
df3

In [ ]:
pd.merge(df1a, df3, left_index = True, right_on = 'name')

### Specifying Set Arithmetic for Joins

In [ ]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns = ['name', 'food'])

df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])

In [ ]:
df6

In [ ]:
df7

In [ ]:
pd.merge(df6, df7)

Here, we have merged two datasets that have only a single “name” entry in common: Mary.

**Inner Join**

By default, the result contains the *intersection* of the two sets of inputs; this is what is known as an *inner join*.

We can specify this explicitly using the how keyword, which defaults to `inner`:

In [ ]:
pd.merge(df6, df7, how = 'inner')

Other options for the how keyword are `outer`, `left`, and `right`.

**Outer Join**

An outer join returns a join over the union of the input columns, and fills in all missing values with NAs:

In [ ]:
df6

In [ ]:
df7

In [ ]:
pd.merge(df6, df7, how = 'outer')

**Left and Right Joins**

The left join and right join return join over the left entries and right entries, respectively. For example:

In [ ]:
df6

In [ ]:
df7

In [ ]:
pd.merge(df6, df7, how = 'left')

In [ ]:
pd.merge(df6, df7, how = 'right')

<img src="https://docs.trifacta.com/download/attachments/160412683/JoinVennDiagram.png?version=1&modificationDate=1596167437085&api=v2" width=300 height = 230 />

## GroupBy Operations: Split, Apply, Combine

<img src="https://jakevdp.github.io/figures/split-apply-combine.svg" width=600 height = 600 />

- The *split* step involves breaking up and grouping a DataFrame depending on the value of the specified key.
- The *apply* step involves computing some function, usually an aggregate, transformation, or filtering, within the individual groups.
- The *combine* step merges the results of these operations into an output array.

In [ ]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

We can compute the most basic split-apply-combine operation with the `groupby()`
method of DataFrames, passing the name of the desired key column:

In [ ]:
df.groupby('key')

Notice that what is returned is not a set of `DataFrames`, but a `DataFrameGroupBy`
object.

To produce a result, we can apply an aggregate to this `DataFrameGroupBy` object, which will perform the appropriate apply/combine steps to produce the desired result:

In [ ]:
df.groupby('key').sum()

The `sum()` method is just one possibility here; you can apply virtually any common Pandas or NumPy aggregation function, as well as virtually any valid `DataFrame` operation

### GroupBy on `athlete_events.csv` dataset

Load data from a csv file

In [ ]:
filename = 'athlete_events.csv'
df = pd.read_csv(filename)
df.head()

In the following code, we will group the rows in the dataframe into the possible values in the 'Games' column, like '1896 Summer', '1900 Summer', etc.

**Iteration Over Groups**

The `GroupBy` object supports direct iteration over the groups, returning each group as a Series or DataFrame:

In [ ]:
games_gb = df.groupby('Games')

# Loop though the GroupBy object, printing the key and the dataframe
for group_key, group_df in games_gb:
    print('The group key:', group_key) # Print the key (for example, '1984 Summer')
    print('The related df:')           # Print the values (all the rows associated with the key)
    display(group_df)

We could just get one key by using the `.get_group(key)` method.

In [ ]:
key = '1984 Summer'
summer84_df = games_gb.get_group(key)
display(summer84_df.head())
summer84_df.shape

We could look at the 'keys' and see how many rows are associated with each.

In [ ]:
games_gb.size()

We can also go multiple levels deep. For example, group first by 'Games', then by 'Sport', then by 'Sex'

In [ ]:
complex_gb = df.groupby(['Games','Sport','Sex'])
complex_gb.size()

Using multiple level groupby objects, if we want to get a specific dataframe, we have to call get_group by using a tuple.

In [ ]:
t = ('2016 Summer','Weightlifting','F')         # Create a Tuple. It has to match all the group by columns.
weightlifting_2016_df = complex_gb.get_group(t) # Call the get_group() method. This returns a DataFrame
weightlifting_2016_df.head()

You can get basic statistics on each groupby category using the `.agg()` method.

In [ ]:
# Setup a new groupby object for 'NOC' and 'Sex'
noc_gender_gb = df.groupby(['NOC','Sex'])

# Call the agg() method, applying ['min','mean','max','count']. These only work on number columns
noc_gender_df = noc_gender_gb.agg(['min','mean','max','count']) # This returns a DataFrame with 'NOC' as the index
noc_gender_df.head()

In [ ]:
# From this big DataFrame, let's only look at the 'Age' column
noc_gender_df['Age'].tail(10)

In [ ]:
# From this big DataFrame, let's only look at 'USA' and the 'Age' column
# Since 'NOC' is the index, we can use the loc[] method to get just the index 'USA'
noc_gender_df.loc['USA','Age']

In [ ]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                  columns = ['key', 'data1', 'data2'])
df

In [ ]:
my_groups = df.groupby('key')
display(my_groups.get_group('A'))
display(my_groups.get_group('B'))
display(my_groups.get_group('C'))

Another useful pattern is to pass a dictionary mapping column names to operations
to be applied on that column:

In [ ]:
my_groups_df = my_groups.agg({'data1': 'min', 'data2': 'max'})
my_groups_df